In [18]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness 

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [20]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://pytorch.org/docs/stable/index.html",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [21]:
text_documents

[Document(metadata={'source': 'https://pytorch.org/docs/stable/index.html'}, page_content='')]

In [31]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()

In [32]:
docs

[Document(metadata={'source': 'attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\n

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:15]

[Document(metadata={'source': 'attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\n

In [35]:
documents

[Document(metadata={'source': 'attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\n

In [58]:
import faiss
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
import PyPDF2  # For PDF text extraction
from sentence_transformers import SentenceTransformer
import pickle

# Define the Sentence Transformer model
model_name = "all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)  # Directly use the SentenceTransformer

# Get the embedding dimension from the model
embedding_dim = embedding_model.encode(["test"]).shape[1]

# Initialize the FAISS vector store (empty, to start adding multiple files' embeddings)
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Initialize an empty list to hold document IDs (for later mapping)
document_ids = []
document_contents = {}  # This will hold the document content (id -> content)

# List of files or documents to process
documents = ["attention.pdf", "speech.txt", "itemInvoiceDownload.pdf"]  # Example files

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()  # Extract text from each page
    return text

# Process each document
for doc_index, doc_path in enumerate(documents):
    # Extract text based on file type
    if doc_path.endswith(".pdf"):
        content = extract_text_from_pdf(doc_path)  # Extract text from PDF
    elif doc_path.endswith(".txt"):
        with open(doc_path, 'r') as file:
            content = file.read()  # Read content from .txt file
    
    # Store the content in the document_contents dictionary
    document_contents[str(doc_index)] = content  # Use doc_index as the key for content
    
    # Generate embeddings for the document
    embeddings = embedding_model.encode([content])  # Directly use the SentenceTransformer's encode method
    
    # Add embeddings to the FAISS index
    faiss_index.add(embeddings.astype("float32"))  # FAISS expects float32 type
    
    # Track document IDs for mapping (here using simple doc_index as ID)
    document_ids.append(str(doc_index))  # You can customize this ID if needed

# Optionally, save the document embeddings as well (for later retrieval)
with open('document_ids.pkl', 'wb') as f:
    pickle.dump(document_ids, f)

# Save the document contents to a pickle file
with open('document_contents.pkl', 'wb') as f:
    pickle.dump(document_contents, f)

# Save the FAISS index to a file for later use
faiss.write_index(faiss_index, 'combined_embeddings.index')

print("Document embeddings and contents have been saved successfully.")


Document embeddings and contents have been saved successfully.


In [60]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle

# Step 1: Load the FAISS index from the saved file
faiss_index = faiss.read_index('combined_embeddings.index')

# Step 2: Load the document IDs (used to map back to the documents)
with open('document_ids.pkl', 'rb') as f:
    document_ids = pickle.load(f)

# Step 3: Load the actual document content (text corresponding to each embedding)
with open('document_contents.pkl', 'rb') as f:
    document_contents = pickle.load(f)

# Step 4: Initialize the SentenceTransformer model (same model used for embedding generation)
model_name = "all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)

# Normalize the embeddings for FAISS index (optional but can help with cosine similarity searches)
def normalize_embeddings(embeddings):
    return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Step 5: Define a function to perform similarity search and answer the query
def answer_query(query, k=1):  # Return the best match (k=1)
    # Step 6: Generate the embedding for the query
    query_embedding = embedding_model.encode([query])

    # Normalize the query embedding (optional)
    query_embedding = normalize_embeddings(query_embedding)

    # Step 7: Perform the similarity search in the FAISS index
    D, I = faiss_index.search(np.array(query_embedding).astype('float32'), k)  # D is the distances, I is the indices

    # Step 8: Retrieve the most similar document (best match)
    best_doc_id = document_ids[I[0][0]]  # Get the ID of the most similar document
    best_doc_content = document_contents.get(best_doc_id, "Content not found")  # Retrieve the exact content

    return best_doc_content  # Return only the content of the best matching document

# Example user query
query = "what is invoice number?"

# Step 9: Get the answer to the user's query
answer = answer_query(query, k=1)  # Return the best matching document only

# Step 10: Print the answer
print(answer)


Tax Invoice
  I2724SA001279703 Invoice Number:   9470404518 PacketID:
  1250365-4138183-1174703 Order Number:   28 Oct 2023 Invoice Date:
  Inter-State Nature of Transaction:   28 Oct 2023 Order Date:
  UTTAR PRADESH Place of Supply:   Goods Nature of Supply:
Bill to / Ship to:
Deepanshu 
35/11 safed colony juhi kidwai nagar   Juhi 
Colony  Kanpur  -  208014  UP,  India  Customer Type: Unregistered
Bill From: Ship From:
Sane Retails Private Limited 
Ksquare Industrial Park, Warehouse 4, Before Padgha Toll naka,, 
Nashik-Mumbai Highway, Near Pushkar Mela Hotel Rahul 
Narkhede, Padgha-Bhiwandi, Thane, MAHARASHTRA-421101Sane Retails Private Limited 
Ksquare Industrial Park, Warehouse 4, Before Padgha Toll naka,, 
Nashik-Mumbai Highway, Near Pushkar Mela Hotel Rahul 
Narkhede, Padgha-Bhiwandi, Thane, MAHARASHTRA-421101
  GSTIN Number: 27AAXCS0974R1Z3   
QtyGross 
AmountDiscountOther
ChargesTaxable
AmountCGSTSGST/ 
UGSTIGST CessTotal Amount
CTNABDST81338390 - Cortina Cream-Coloured & Orange